## 1.3 변수 소개

TensorFlow **변수**는 프로그램이 조작하는 공유 영구 상태를 나타내는 권장 방법입니다. 이 가이드는 TensorFlow에서 `tf.Variable` 인스턴스를 작성, 업데이트 및 관리하는 방법을 설명합니다.

변수는 `tf.Variable` 클래스를 통해 생성 및 추적됩니다. `tf.Variable`은 ops를 실행하여 값을 변경할 수 있는 텐서를 나타냅니다. 특정 ops를 사용하면 이 텐서의 값을 읽고 수정할 수 있습니다. `tf.keras`와 같은 상위 수준의 라이브러리는 `tf.Variable`을 사용하여 모델 매개변수를 저장합니다. 

### 설정

In [1]:
import tensorflow as tf

In [2]:
import logging    # 경고 출력 금지
logging.getLogger('tensorflow').disabled = True

### 변수 만들기

변수를 작성하려면 초기값을 제공합니다. `tf.Variable`은 초기화 값과 같은 `dtype`을 갖습니다.

In [3]:
my_tensor = tf.constant([[1.0, 2.0], [3.0, 4.0]])
my_variable = tf.Variable(my_tensor)

bool_variable = tf.Variable([False, False, False, True])
complex_variable = tf.Variable([5 + 4j, 6 + 1j])

변수는 텐서처럼 보이고 작동하며, 실제로 `tf.Tensor`에서 지원되는 데이터 구조입니다. 텐서와 마찬가지로, `dtype`과 형상을 가지며 NumPy로 내보낼 수 있습니다.

In [4]:
print("Shape: ",my_variable.shape)
print("DType: ",my_variable.dtype)
print("As NumPy: ", my_variable.numpy)
print("NumPy: ", my_variable.numpy())

Shape:  (2, 2)
DType:  <dtype: 'float32'>
As NumPy:  <bound method BaseResourceVariable.numpy of <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>>


변수를 재구성할 수는 없지만, 대부분의 텐서 연산은 예상대로 변수에 대해 작동합니다.

In [51]:
print("A variable:",my_variable)
print("\nViewed as a tensor:", tf.convert_to_tensor(my_variable))
print("\nIndex of highest value:", tf.argmax(my_variable))

print("\nCopying and reshaping: ", tf.reshape(my_variable, ([1,4])))

A variable: <tf.Variable 'Variable:0' shape=(2, 2) dtype=float32, numpy=
array([[1., 2.],
       [3., 4.]], dtype=float32)>

Viewed as a tensor: tf.Tensor(
[[1. 2.]
 [3. 4.]], shape=(2, 2), dtype=float32)

Index of highest value: tf.Tensor([1 1], shape=(2,), dtype=int64)

Copying and reshaping:  tf.Tensor([[1. 2. 3. 4.]], shape=(1, 4), dtype=float32)


위에서 언급했듯이, 변수는 텐서에 의해 지원됩니다. `tf.Variable.assign`을 사용하여 텐서를 재할당할 수 있습니다. `assign`을 호출해도 (일반적으로) 새로운 텐서를 할당하지 않고, 대신 기존 텐서의 메모리가 재사용됩니다.

In [52]:
a = tf.Variable([2.0, 3.0])
a.assign([1, 2]) 

try:
  a.assign([1.0, 2.0, 3.0])
except Exception as e:
  print(f"{type(e).__name__}: {e}")

ValueError: Cannot assign to variable Variable:0 due to variable shape (2,) and value shape (3,) are incompatible


연산에서 텐서와 같은 변수를 사용하는 경우, 일반적으로 지원 텐서에서 작동합니다.

기존 변수에서 새 변수를 만들면 지원 텐서가 복제됩니다. 두 변수는 같은메모리를 공유하지 않습니다.

In [53]:
a = tf.Variable([2.0, 3.0])
b = tf.Variable(a)
a.assign([5, 6])

print(a.numpy())
print(b.numpy())

print(a.assign_add([2,3]).numpy())  
print(a.assign_sub([7,9]).numpy())  

[5. 6.]
[2. 3.]
[7. 9.]
[0. 0.]


### 수명 주기, 이름 지정 및 감시

파이썬 기반 TensorFlow에서 `tf.Variable` 인스턴스는 다른 Python 객체와 같은 수명 주기를 갖습니다. 변수에 대한 참조가 없으면 자동으로 할당이 해제됩니다.

변수를 추적하고 디버그하는 데 도움이 되는 변수의 이름을 지정할 수도 있습니다. 두 변수에 같은 이름을 지정할 수 있습니다.

In [54]:
a = tf.Variable(my_tensor, name="Mark")
b = tf.Variable(my_tensor + 1, name="Mark")

print(a == b)

tf.Tensor(
[[False False]
 [False False]], shape=(2, 2), dtype=bool)


모델을 저장하고 로드할 때 변수 이름이 유지됩니다. 기본적으로 모델의 변수는 고유한 변수 이름이 자동으로 지정되므로 원치 않는 한 직접 할당할 필요가 없습니다.

변수는 구별을 위해 중요하지만, 일부 변수는 구별할 필요가 없습니다. 생성 시 `trainable`을 false로 설정하여 변수의 그래디언트를 끌 수 있습니다. 그래디언트가 필요하지 않은 변수의 예는 훈련 단계 카운터입니다.

In [55]:
step_counter = tf.Variable(1, trainable=False)

### 변수 및 텐서 배치하기

더 나은 성능을 위해 TensorFlow는 `dtype`과 호환되는 가장 빠른 기기에 텐서 및 변수를 배치하려고 시도합니다. 이는 대부분의 변수가 GPU(사용 가능한 경우)에 배치됨을 의미합니다.

그러나 재정의할 수 있습니다. 다음 코드 조각에서는 GPU가 사용 가능한 경우에도 부동 텐서와 변수를 CPU에 배치할 수 있습니다. 

참고: 수동 배치도 가능하지만, [분배 전략](distributed_training)을 사용하면 계산을 최적화하는 더 편리하고 확장 가능한 방법이 될 수 있습니다.

In [56]:
with tf.device('CPU:0'):
    a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
    b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
    c = tf.matmul(a, b)

print(c)

tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


한 기기에서 변수 또는 텐서의 위치를 설정하고 다른 기기에서 계산을 수행할 수 있습니다. 이 경우, 기기 간에 데이터를 복사해야 하므로 지연이 발생합니다.

GPU 작업자가 여러 개이지만 변수의 사본이 하나만 필요한 경우에 수행할 수 있습니다.

In [57]:
with tf.device('CPU:0'):
  a = tf.Variable([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
  b = tf.Variable([[1.0, 2.0, 3.0]])

with tf.device('GPU:0'):
  k = a * b

print(k)

tf.Tensor(
[[ 1.  4.  9.]
 [ 4. 10. 18.]], shape=(2, 3), dtype=float32)
